<a href="https://colab.research.google.com/github/ekaropolus/recommender_and_graph_spp/blob/main/Extractor_Neo4j_ASDI_Sustainable_Public_Policies_MX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install neo4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 95 kB 1.6 MB/s 
  Created wheel for neo4j: filename=neo4j-4.4.5-py3-none-any.whl size=123291 sha256=afeac510ca351ca4d73718e300639e16a5ce26b93fd8dd01342189071e5ce807
  Stored in directory: /root/.cache/pip/wheels/73/62/2d/6532ffe9b1fdc65f04e7461279cda78fccaa67cf3c706e6e3d
Successfully built neo4j


In [129]:
from neo4j import GraphDatabase
import logging
from neo4j.exceptions import ServiceUnavailable

class App:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        # Don't forget to close the driver connection when you are finished with it
        self.driver.close()

    def find_public_policy(self):
        with self.driver.session(database="neo4j") as session:
            result = session.read_transaction(self._find_and_return_public_policy)
            return result

    @staticmethod
    def _find_and_return_public_policy(tx):
        query = (
            "MATCH (n) "
            "WHERE n.Country = 'MX' "
            "RETURN n.PublicPolicyId as id, n.Level as group"
        )
        result = tx.run(query)
        return [(row["id"],row["group"]) for row in result]

    def find_public_policy_relations(self):
        with self.driver.session(database="neo4j") as session:
            result = session.read_transaction(self._find_and_return_public_policy_relations)
            return result

    @staticmethod
    def _find_and_return_public_policy_relations(tx):
        query = (
            "MATCH (n)-[r:DIVIDED_IN]->(m) "
            "WHERE n.Country = 'MX' "
            "RETURN n.PublicPolicyId as source, m.PublicPolicyId as target, (n.Level + m.Level) as value"
        )
        result = tx.run(query)
        return [(row["source"],row["target"],row["value"]) for row in result]



In [130]:
# Aura queries use an encrypted connection using the "neo4j+s" URI scheme
uri = "neo4j+s://2c2e2321.databases.neo4j.io"
user = "neo4j"
password = "kPJiOCU2ekZYW7159DH_2-4-638msHWntxaWbRkxGVQ"
app = App(uri, user, password)


In [132]:
li_pp_result = app.find_public_policy()
li_pp_relations_result = app.find_public_policy_relations()

In [ ]:
for row in li_pp_result:
  print(row)

for row in li_pp_relations_result:
  print(row)

In [134]:
di_dummy = {}
li_nodes = []
for row in li_pp_result:
  di_dummy = {}
  di_dummy["id"] = row[0]
  di_dummy["group"] = row[1]
  li_nodes.append(di_dummy)

li_relations = []
for row in li_pp_relations_result:
  di_dummy = {}
  di_dummy["source"] = row[0]
  di_dummy["target"] = row[1]
  di_dummy["value"] = row[2]
  li_relations.append(di_dummy)

In [ ]:
li_nodes

In [ ]:
li_relations


In [137]:
di_pp_view = {}

In [138]:
di_pp_view["nodes"] = li_nodes
di_pp_view["links"] = li_relations

In [ ]:
di_pp_view

In [140]:
import pickle


In [142]:
f = open("dict.txt","w")

# write file
f.write( str(di_pp_view) )

# close file
f.close()

In [143]:
app.close()